In [6]:
# uncomment to install the necessary libraries in Colab
!pip install -q diffusers transformers accelerate trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 10.5 MB/s eta 0:00:00 0:00:01


In [13]:
pip install diffusers transformers accelerate torch gradio safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 33.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 96.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:

prior_pipeline = DiffusionPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-1-prior",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

pipeline = DiffusionPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-1",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

# Load Shap-E Img2Img

In [ ]:

shape_pipe = ShapEImg2ImgPipeline.from_pretrained(
    "openai/shap-e-img2img",
    torch_dtype=torch.float16
).to("cuda")

In [ ]:
def generate_3d(prompt):
    # Step 1: Generate image from prompt using Kandinsky
    image_embeds, negative_image_embeds = prior_pipeline(prompt, guidance_scale=1.0).to_tuple()
    image = pipeline(prompt, image_embeds=image_embeds, negative_image_embeds=negative_image_embeds).images[0]

    # Save intermediate image
    image_path = "generated_image.png"
    image.save(image_path)

    # Step 2: Convert image to 3D rotating gif using Shap-E
    input_img = Image.open(image_path).resize((256, 256))
    result = shape_pipe(
        input_img,
        guidance_scale=3.0,
        num_inference_steps=32,
        frame_size=128
    )

    gif_path = "output_3d.gif"
    export_to_gif(result.images[0], gif_path)

    return gif_path

# Gradio UI

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Prompt to 3D GIF Generator")
    gr.Markdown("Generate a 3D-like rotating object from text using Kandinsky + Shap-E")
    
    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(lines=2, label="Enter your prompt", placeholder="A red apple, 3D")
            generate_btn = gr.Button("Generate")
        with gr.Column():
            output_gif = gr.Image(label="3D Output", type="filepath")
    
    generate_btn.click(fn=generate_3d, inputs=prompt_input, outputs=output_gif)

demo.launch(share=True)

In [ ]:
import torch
from diffusers import DiffusionPipeline, ShapEImg2ImgPipeline
from diffusers.utils import export_to_gif
from PIL import Image
import gradio as gr
import os

# Load Kandinsky 2.1 pipelines
prior_pipeline = DiffusionPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-1-prior",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

pipeline = DiffusionPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-1",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

# Load Shap-E Img2Img pipeline
shape_pipe = ShapEImg2ImgPipeline.from_pretrained(
    "openai/shap-e-img2img",
    torch_dtype=torch.float16
).to("cuda")

def generate_3d(prompt):
    # === STEP 1: Generate 2D image from text ===
    image_embeds, negative_image_embeds = prior_pipeline(prompt, guidance_scale=1.0).to_tuple()
    image = pipeline(prompt, image_embeds=image_embeds, negative_image_embeds=negative_image_embeds).images[0]

    # Save intermediate 2D image
    image_path = "generated_2d.png"
    image.save(image_path)

    # === STEP 2: Convert 2D image to rotating 3D-style gif ===
    input_image = Image.open(image_path).resize((256, 256))
    images = shape_pipe(
        input_image,
        guidance_scale=3.0,
        num_inference_steps=64,
        frame_size=256
    ).images

    gif_path = "generated_3d.gif"
    export_to_gif(images[0], gif_path)

    return gif_path

# === Gradio UI ===
interface = gr.Interface(
    fn=generate_3d,
    inputs=gr.Textbox(label="Enter a prompt (e.g., A cheeseburger, white background, 3D)"),
    outputs=gr.Image(type="filepath", label="3D Output GIF"),
    title="Text-to-3D Generator using Kandinsky 2.1 & Shap-E",
    description="Enter a prompt to generate a 3D rotating object animation."
)

interface.launch()
